In [30]:
from gurobipy import *

M = Model()
# number of players in the team
n = 14
# player strength list, either 2.5 or 3.0 for each player
N = [2.5, 3.0, 3.0, 2.5, 3.0, 2.5, 3.0, 3.0, 2.5, 2.5, 2.5, 3.0, 2.5, 2.5]
# opponent strength list, either 2.5 or 3.0 for each court position
N0 = [3.0, 3.0, 3.0, 3.0, 3.0, 2.5, 3.0, 3.0]
# number of games in the tournament
m = 8
# number of slots for a player to play in per game
o = 8
# minimum number of games each player must play
p = 3
# availability matrix. 1 if the ith player can make the jth game, 0 if they cannot
A = [[1 for j in range(m)] for i in range(n)] 
# preference matrix. 1 if the ith player prefers the jth game and kth slot, -1 if they prefer not to, 0 if no preference
P = [[[0 for k in range(o)] for j in range(m)] for i in range(n)] 
for j in range(0,m):
    P[1][j][6] = 1 #test preferences - TODO: add actual player preferences
    P[2][j][7] = 1
# 1 if a player i plays in game j on the kth court, 0 if they do not
x = M.addVars(n, m, o, vtype=GRB.BINARY)

M.update()

for i in range(0,n):
    M.addConstrs(x.sum(i, j, '*') <= A[i][j] for j in range(0,m)) #players can only play one court in a match, 
                                                                  #or none if they are not available
    M.addConstr(x.sum(i, '*', '*'), GRB.GREATER_EQUAL, p) #players must play at least p games in the season
for k in range(0,o):
    M.addConstrs(x.sum('*', j, k) == 1 for j in range(0,m)) #all courts must have exactly one player
    
M.update()

expr = LinExpr(-188)
opponentSum = 0;
for k in range(0,o):
    for j in range(0,m):
        opponentSum += N0[j]
        for i in range(0,n):
            expr.add(x[i,j,k], N[i]) # "overall team strength" - TODO: rework condition
            expr.add(x[i,j,k], P[i][j][k]) # preferences
            
M.setObjective(expr, GRB.MAXIMIZE)

M.optimize()


Optimize a model with 190 rows, 896 columns and 2688 nonzeros
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -11.5000000
Presolve time: 0.01s
Presolved: 190 rows, 896 columns, 2688 nonzeros
Variable types: 0 continuous, 896 integer (896 binary)

Root relaxation: objective 8.000000e+00, 383 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%     -    0s

Explored 0 nodes (383 simplex iterations) in 0.06 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 8 -11.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.00

In [31]:
for j in range(0,m):
    print(P[1][j][6])
    print(x[1,j,6].x)

1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
1
1.0
